In [1]:
import os
import sys
sys.path.append("/home/pervinco/BKAI-IGH-NeoPolyp")

import cv2
import yaml
import random
import numpy as np
import albumentations as A
import matplotlib.pyplot as plt

from tqdm import tqdm
from glob import glob
from data.data_preprocess import spatially_exclusive_pasting, train_img_mask_transform

In [2]:
def encode_mask(mask):  
    hsv_mask = cv2.cvtColor(mask, cv2.COLOR_BGR2HSV)
    
    lower1 = np.array([0, 100, 20])
    upper1 = np.array([10, 255, 255])
    
    lower2 = np.array([160,100,20])
    upper2 = np.array([179,255,255])

    lower_mask = cv2.inRange(hsv_mask, lower1, upper1)
    upper_mask = cv2.inRange(hsv_mask, lower2, upper2)
    red_mask = lower_mask + upper_mask
    red_mask[red_mask != 0] = 2 * 127  # converting 2 to 8-bit scale

    green_mask = cv2.inRange(hsv_mask, (36, 25, 25), (70, 255,255))
    green_mask[green_mask != 0] = 1 * 127  # converting 1 to 8-bit scale
    full_mask = cv2.bitwise_or(red_mask, green_mask)
    full_mask = full_mask.astype(np.uint8)

    return full_mask

def decode_mask(encoded_mask):
    """Decode an 8-bit mask back to original values."""
    decoded_mask = np.zeros_like(encoded_mask)
    decoded_mask[encoded_mask == 127] = 1
    decoded_mask[encoded_mask == 254] = 2
    return decoded_mask

In [3]:
train_transform = A.Compose([A.CLAHE(p=0.4),
                             A.RandomBrightnessContrast(p=0.4),
                             A.Rotate(limit=90, border_mode=0, p=0.6),
                             A.HorizontalFlip(p=0.5),
                             A.VerticalFlip(p=0.5),
                             A.RandomGamma (gamma_limit=(70, 130), eps=None, always_apply=False, p=0.2),
                             A.RGBShift(p=0.3, r_shift_limit=10, g_shift_limit=10, b_shift_limit=10),
                             A.OneOf([A.Blur(), A.GaussianBlur(), A.GlassBlur(), A.MotionBlur(), A.GaussNoise(), A.Sharpen(), A.MedianBlur(), A.MultiplicativeNoise()]),
                             A.CoarseDropout(p=0.2, max_height=35, max_width=35, fill_value=255, mask_fill_value=0),
                             A.ShiftScaleRotate(p=0.45, border_mode=cv2.BORDER_CONSTANT, shift_limit=0),
                             A.RandomCrop(width=640, height=485, p=0.4),
                             A.Resize(width=1280, height=990, p=1)])

In [4]:
with open("/home/pervinco/BKAI-IGH-NeoPolyp/config.yaml", "r") as f:
    config = yaml.safe_load(f)

In [5]:
data_dir = config["data_dir"]
image_dir = f"{data_dir}/train"
mask_dir = f"{data_dir}/train_gt"

color_dir = f"{data_dir}/color_txt"
color_files = sorted(glob(f"{color_dir}/*.txt"))

In [6]:
total = {}
for color_file in color_files:
    name = color_file.split('/')[-1].split('.')[0]
    with open(color_file, 'r') as f:
        files = [x.strip() for x in f.readlines()]

    total.update({name : files})

for name, files in total.items():
    print(name, len(files))

green 257
red 694
rng 49


In [7]:
if not os.path.isdir(f"{data_dir}/augment"):
    os.makedirs(f"{data_dir}/augment/images")
    os.makedirs(f"{data_dir}/augment/masks")
    os.makedirs(f"{data_dir}/augment/encoded_masks")

In [8]:
# for name, files in total.items():
#     if name == "green":
#         num_iter = 6
#     elif name == "rng":
#         num_iter = 4
#     else:
#         num_iter = 2

#     for idx in tqdm(range(len(files))):
#         file = files[idx]
#         for it in range(num_iter):
#             image_file = f"{image_dir}/{file}.jpeg"
#             mask_file = f"{mask_dir}/{file}.jpeg"

#             image = cv2.imread(image_file)
#             mask = cv2.imread(mask_file)

#             paste_image, paste_mask = spatially_exclusive_pasting(image, mask, alpha=random.uniform(config["spatial_alpha"], config["spatial_alpha"] + 0.2))
#             result_image, result_mask = train_img_mask_transform(train_transform, paste_image, paste_mask)
            
#             result_mask = encode_mask(result_mask)
            
#             overlay = decode_mask(result_mask)
#             overlay = cv2.cvtColor(overlay, cv2.COLOR_BGR2RGB)

#             cv2.imwrite(f"{data_dir}/augment/images/{file}_{it}.jpeg", result_image)
#             cv2.imwrite(f"{data_dir}/augment/masks/{file}_{it}.png", result_mask)
#             cv2.imwrite(f"{data_dir}/augment/overlay/{file}_{it}.png", overlay)

In [9]:
for name, files in total.items():
    for idx in tqdm(range(len(files))):
        file = files[idx]
    # for file in files:
        image_file = f"{image_dir}/{file}.jpeg"
        mask_file = f"{mask_dir}/{file}.jpeg"

        original_image = cv2.imread(image_file)
        original_mask = cv2.imread(mask_file)

        encoded_mask = encode_mask(original_mask)

        cv2.imwrite(f"{data_dir}/augment/images/{file}.jpeg", original_image)
        cv2.imwrite(f"{data_dir}/augment/masks/{file}.jpeg", original_mask)
        cv2.imwrite(f"{data_dir}/augment/encoded_masks/{file}.jpeg", encoded_mask)        

        result_image, result_mask = spatially_exclusive_pasting(original_image, original_mask, alpha=random.uniform(config["spatial_alpha"], config["spatial_alpha"] + 0.2))        
        result_encoded_mask = encode_mask(result_mask)
        # print(np.unique(result_encoded_mask))

        cv2.imwrite(f"{data_dir}/augment/images/{file}_aug.jpeg", result_image)
        cv2.imwrite(f"{data_dir}/augment/masks/{file}_aug.jpeg", result_mask)
        cv2.imwrite(f"{data_dir}/augment/encoded_masks/{file}_aug.jpeg", result_encoded_mask)

100%|██████████| 49/49 [00:03<00:00, 14.25it/s]


In [12]:
if not os.path.isdir(f"{data_dir}/augment/check"):
    os.makedirs(f"{data_dir}/augment/check")

mask_files = sorted(glob(f"{data_dir}/augment/encoded_masks/*"))
for mask_file in mask_files:
    file_name = mask_file.split('/')[-1].split('.')[0]
    mask = cv2.imread(mask_file, cv2.IMREAD_GRAYSCALE)
    colored_mask = decode_mask(mask)
    print(np.unique(colored_mask))
    
    cv2.imwrite(f"{data_dir}/augment/check/{file_name}.jpeg", colored_mask)

[0 2]
[0 2]
[0 2]
[0 2]
[0 2]
[0 2]
[0 1]
[0 1]
[0 1]
[0 1]
[0 2]
[0 2]
[0 2]
[0 2]
[0 2]
[0 2]
[0 2]
[0 2]
[0 2]
[0 2]
[0 2]
[0 2]
[0 2]
[0 2]
[0 1]
[0 1]
[0 2]
[0 2]
[0 1 2]
[0 1 2]
[0 2]
[0 2]
[0 2]
[0 2]
[0 2]
[0 2]
[0 2]
[0 2]
[0 2]
[0 2]
[0 2]
[0 2]
[0 2]
[0 2]
[0 1]
[0 1]
[0 1 2]
[0 1 2]
[0 1]
[0 1]
[0 2]
[0 2]
[0 2]
[0 2]
[0 1]
[0 1]
[0 2]
[0 2]
[0 2]
[0 2]
[0 1 2]
[0 1 2]
[0 2]
[0 2]
[0 1]
[0 1]
[0 2]
[0 2]
[0 2]
[0 2]
[0 2]
[0 2]
[0 1]
[0 1]
[0 2]
[0 2]
[0 2]
[0 2]
[0 2]
[0 2]
[0 1]
[0 1]
[0 2]
[0 2]
[0 2]
[0 2]
[0 2]
[0 2]
[0 2]
[0 2]
[0 1]
[0 1]
[0 2]
[0 2]
[0 2]
[0 2]
[0 2]
[0 2]
[0 2]
[0 2]
[0 1]
[0 1]
[0 2]
[0 2]
[0 1]
[0 1]
[0 1]
[0 1]
[0 2]
[0 2]
[0 2]
[0 2]
[0 1 2]
[0 1 2]
[0 2]
[0 2]
[0 2]
[0 2]
[0 1]
[0 1]
[0 2]
[0 2]
[0 2]
[0 2]
[0 2]
[0 2]
[0 2]
[0 2]
[0 2]
[0 2]
[0 2]
[0 2]
[0 2]
[0 2]
[0 1]
[0 1]
[0 2]
[0 2]
[0 1]
[0 1]
[0 2]
[0 2]
[0 1]
[0 1]
[0 2]
[0 2]
[0 1]
[0 1]
[0 2]
[0 2]
[0 2]
[0 2]
[0 2]
[0 2]
[0 1]
[0 1]
[0 2]
[0 2]
[0 2]
[0 2]
[0 1]
[0 1]
[0 2]
[0 2]
